# Детекция отсутствия изоляторов в гирлянде изоляторов

### Построение датасета

In [2]:
# сразу установим все нужные библиотеки
!pip install pycocotools
!pip install faster-coco-eval
!pip install torchmetrics
!pip install torchmetrics[detection]==0.8.0

  Obtaining dependency information for pycocotools from https://files.pythonhosted.org/packages/ba/64/0451cf41a00fd5ac4501de4ea0e395b7d909e09d665e56890b5d3809ae26/pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
Using cached pycocotools-2.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (426 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 6.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: torchmetrics
    Found existing installation: torchmetrics 1.2.1
    Uninstalling torchmetrics-1.2.1:
      Successfully uninstalled torchmetrics-1.2.1


In [3]:
# импортируем нужную нам метрику
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.ops import batched_nms # штука для Non maximum supression

In [6]:
import pandas as pd


# targets = pd.read_csv('/kaggle/input/innopolis-high-voltage-challenge/sample_submission_1.csv')
# targets = targets[['file_name', 'x', 'y', 'w', 'h']]
# targets

In [7]:
import os 
from glob import glob
from torch.utils.data import Dataset
from PIL import Image
import torch


# лоадер для фоток(подаем ссылку, возвращаем изображение)
def _loader(path: str) -> Image.Image:
    '''
        params: path to image
        return Image.Image our image
    '''
    
    img = Image.open(path) # октрываем наш файл jpg по пути и возвращаем
    return img 
    
# класс датасетом для сабмита
class Discs(Dataset):
    def __init__(self, root: str, split: str = 'train', transform=None):
        '''
            params: root - path to file
            params: targ - dataframe with name objects and coordinates bndboxes
            params: split - train/validate mode
            params: transform - image transform
        '''
        
        self.root = os.path.join(root, split) # запишем путь c режимом в root
        
        self.images = glob(os.path.join(root, '*')) # открываем файл по пути и заглядываем в изображения
#         self.images.remove('/kaggle/input/innopolis-high-voltage-challenge/sample_submission_1.csv') # удаляем из списка изображений путь с sample_submission
        
#         self.targets = targ # запишем датафрейм
        
#         self.target_bndboxes = {} # словарь вида name: bndboxes
        
#         # пройдемся по длине датафрейма
#         for row in range(self.targets.shape[0]):
#             # если в нашем словаре нет ключа с таким именем то записываем по этому ключу все bndbox'ы
#             if self.targets.iloc[row].file_name[:-2] not in self.target_bndboxes.keys():
#                 self.target_bndboxes[self.targets.iloc[row].file_name[:-2]] = torch.tensor([self.targets.iloc[row, 1:5].values.astype('float64')])
#             # иначе такой ключ есть, добавим в список bndboxe'ов еще один
#             else:
#                 self.target_bndboxes[self.targets.iloc[row].file_name[:-2]] = torch.cat((self.target_bndboxes[self.targets.iloc[row].file_name[:-2]], torch.tensor([self.targets.iloc[row, 1:5].values.astype('float64')])))
        
        self.transform = transform

    def __getitem__(self, item):
        '''
            params: item for return object
        '''
        image1 = _loader(self.images[item])
        
#         label = (self.images[item].split('/')[-1])[:-4] # возьмем из пути к изображению только название
        
        if self.transform is not None:
            image1 = self.transform(image1)
        
        return image1
    
    def __len__(self):
        return len(self.images)

In [8]:
from torchvision import transforms as T


dics = Discs(r'/kaggle/input/innopolis-high-voltage-challenge', 'test', T.ToTensor())


dics[14]

tensor([[[0.5176, 0.5216, 0.5255,  ..., 0.5294, 0.5373, 0.5059],
         [0.5765, 0.5529, 0.5451,  ..., 0.4863, 0.5059, 0.4667],
         [0.5608, 0.5255, 0.5255,  ..., 0.4824, 0.5490, 0.5451],
         ...,
         [0.2824, 0.3686, 0.3804,  ..., 0.7373, 0.7255, 0.6275],
         [0.2745, 0.4000, 0.4392,  ..., 0.6824, 0.6745, 0.5882],
         [0.3059, 0.4431, 0.4902,  ..., 0.6510, 0.6667, 0.6118]],

        [[0.4588, 0.4627, 0.4667,  ..., 0.4824, 0.4902, 0.4588],
         [0.5176, 0.4941, 0.4863,  ..., 0.4392, 0.4588, 0.4196],
         [0.5020, 0.4667, 0.4667,  ..., 0.4353, 0.5020, 0.4980],
         ...,
         [0.2196, 0.3059, 0.3176,  ..., 0.6118, 0.6000, 0.5020],
         [0.2196, 0.3451, 0.3843,  ..., 0.5569, 0.5373, 0.4510],
         [0.2510, 0.3882, 0.4353,  ..., 0.5137, 0.5294, 0.4745]],

        [[0.3765, 0.3804, 0.3843,  ..., 0.4196, 0.4275, 0.3961],
         [0.4353, 0.4118, 0.4039,  ..., 0.3765, 0.3961, 0.3569],
         [0.4196, 0.3843, 0.3765,  ..., 0.3804, 0.4471, 0.

In [9]:
from torch.utils.data import DataLoader


test_loader = DataLoader(dics, batch_size=8, pin_memory=True, num_workers=8)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
from torchvision.ops import box_convert

# из пути в изображение
def _loader(path: str) -> Image.Image:
    
    img = Image.open(path)
    
    return img

# из пути в словарь тензоров с координатами бокса в формате(xmin, ymin, xmax, ymax)
def _text_loader(path: str) -> list:
    
    text = open(path).read().split('\n')
    
    tensor_coords = torch.tensor([], dtype=torch.float64)

    for i in text:
        if len(i.split()) != 5:
            tensor_coords = torch.cat((tensor_coords, torch.tensor([[1, 1, 2, 2]])))
        else:
            q = ''.join(i)
            tensor_coords = torch.cat((tensor_coords, torch.tensor([list(map(float, q.split()))[1:]]) * 640)) 
    
    
    tensor_coords = box_convert(tensor_coords, in_fmt='cxcywh', out_fmt='xyxy')
    
    return {'boxes': tensor_coords,
           'labels': torch.tensor([1 for i in range(len(tensor_coords))])}
   
# датасет для данных
class Disc(Dataset):
    def __init__(self, root_image: str, root_bndboxes: str, split: str = 'train', transform=None):
        
        self.images = sorted(glob(os.path.join(root_image, '*')))
        
        self.bndboxes = sorted(glob(os.path.join(root_bndboxes, '*')))
        
        self.transform = transform
        
    
    def __getitem__(self, item):
        
        image = _loader(self.images[item])
        
        label = _text_loader(self.bndboxes[item])
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label
    
    def __len__(self):
        return len(self.images)

In [11]:
train_data = Disc(r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train/images', r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train/labels')

In [12]:
# t = _text_loader('/kaggle/input/v1-data/brokeInsulators.v1i.yolov7pytorch/train/labels/11cd2c61db75032b5a6412_jpg.rf.1b2328cafdf73ad1d90cd67f08f34d01.txt')

In [13]:
train_data[10]

(<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x640>,
 {'boxes': tensor([[157.2500, 212.0000, 270.7500, 290.0000],
          [221.5000, 171.0000, 343.5000, 250.0000]], dtype=torch.float64),
  'labels': tensor([1, 1])})

In [14]:
# изобразим наши детекционные прямоугольники на изображении
from torchvision.utils import draw_bounding_boxes
from torchvision import transforms as T
import numpy as np
import matplotlib.pyplot as plt

def show(imgs):
    '''
        params: imgs - our image where we wanna see detection boxes
        return: None
    '''
    to_pil = T.ToPILImage() # функция для перевода в PILImage
    
    if not isinstance(imgs, list): # если изображение не список, переводим в список
        imgs = [imgs]
    
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=(16, 8))
    
    # 
    for i, img in enumerate(imgs):
        img = img.detach() # копия тензора с img, но с отключенными градиентами
        img = to_pil(img) # перевод из тензора в PILImage
        axs[0, i].imshow(np.asarray(img)) # вывод изображения 
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
        
        
def draw_boxes_single_image(image, boxes, labels):
    colors = ['red' for _ in boxes]
    
    show(draw_bounding_boxes(image, boxes, labels, colors=colors, width=5))

In [15]:
from torchvision.ops import box_convert

# t = train_data[6][1]['boxes']

# # conv = box_convert(t, in_fmt='cxcywh', out_fmt='xyxy')
# # conv = conv.unsqueeze(0)

# draw_boxes_single_image(
#     T.PILToTensor()(train_data[6][0]),
#     t,
#     [' 00']
# )

In [16]:
train_dataset = Disc(r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train/images', r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train/labels', 'train', T.ToTensor())
valid_dataset = Disc(r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/valid/images', r'/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/valid/labels', 'valid', T.ToTensor())

In [17]:
train_datasetv = Disc(r'/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/train/images', r'/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/train/labels', 'train', T.ToTensor())
valid_datasetv = Disc(r'/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/valid/images', r'/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/valid/labels', 'valid', T.ToTensor())

In [18]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_dataset, batch_size=8, num_workers=8, pin_memory=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=8, num_workers=8, pin_memory=True, collate_fn=collate_fn)

train_loaderv = DataLoader(train_datasetv, batch_size=8, num_workers=8, pin_memory=True, collate_fn=collate_fn)
valid_loaderv = DataLoader(valid_datasetv, batch_size=8, num_workers=8, pin_memory=True, collate_fn=collate_fn)

In [19]:
from torchvision.models.detection import fasterrcnn_mobilenet_v3_large_320_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


def get_detection_model():
    # нам нужно переобучить модель под наши данные
    model = fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True) 
    
    model.roi_heads.box_predictor = FastRCNNPredictor(1024, 2) # меняем выходной слой, подстроившись под наши данные
    
    return model

In [20]:
# %pip install torchmetrics # отсюда будем импортировать метрику

In [21]:
# !pip install torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.ops import batched_nms # штука для Non maximum supression

def intersection_over_union(dt_bbox, gt_bbox):
    int_x0 = max(dt_bbox[0], gt_bbox[0])
    int_y0 = max(dt_bbox[1], gt_bbox[1])
    
    int_x1 = min(dt_bbox[2], gt_bbox[2])
    int_y1 = min(dt_bbox[3], gt_bbox[3])
    
    intersection = max(int_x1 - int_x0, 0) * max(int_x1 - int_y0, 0)
    dt_area = (dt_bbox[2] - dt_bbox[0]) * (dt_bbox[3] - dt_bbox[1])
    gt_area = (gt_bbox[2] - gt_bbox[0]) * (gt_bbox[3] - gt_bbox[1])
    
    return intersection / (dt_area + gt_area - intersection)


def apply_nms(orig_prediction, iou_thresh=0.3):
    '''
        params: orig_prediction - our prediction
        params: iou_thresh=0.3 - our threshold for nms
    '''
    # передаем все в batched_nms и получаем индексы прямоугольников, которые нужно оставить
    keep = batched_nms(orig_prediction['boxes'], orig_prediction['scores'], orig_prediction['labels'], iou_thresh)
    
    final_prediction = orig_prediction # финальное предсказание
    
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction


@torch.inference_mode()
def visualize(model, batch):
    model.eval()
    
    to_pil = T.ToPILImage()
    to_tensor = T.PILToTensor()
    
    xs, ys = batch
    
    for i, (x, y) in enumerate(zip(xs, ys)):
        # передаем модели список с картинками и получаем список с предсказаниями с боксами, со скорами и с лэйблами
        prediction = {k: v.to(device) for k, v in model([x.to(device)])[0].items()}
        
        # пропускаем через nms чтобы оставить только нужные индексы
        prediction = apply_nms(prediction)
        
        x = to_tensor(to_pil(x.cpu())) 
        
        true_boxes = draw_bounding_boxes(x, boxes=y['boxes'], labels=[str(i) for i in range(len(y['boxes']))], width=5, colors='red')
        predicted_boxes = draw_bounding_boxes(x, boxes=prediction['boxes'], labels=[str(i) for i in range(len(prediction['boxes']))], width=5, colors='red')
        
        fig, ax = plt.subplots(1, 2, figsize=(20, 10), facecolor='white')
        
        ax[0].imshow(to_pil(true_boxes))
        ax[1].imshow(to_pil(predicted_boxes))
        
        ax[0].axis('off')
        ax[1].axis('off')
        
        ax[0].set_title('Target boxes')
        ax[1].set_title('Predicted boxes')
        
        plt.subplots_adjust(wspace=0, hspace=0.1)
        plt.show()
        
        if i >= 9:
            break

In [22]:
from tqdm import tqdm


def train(model, loader, optimizer):
    '''
        params: our model for train
        return: none
    '''
    model.train() # переводим модель в режим обучения
    
    for x, y in tqdm(loader, desc='train'):
        x = list(_.to(device).float() for _ in x) # объект переводим в список из картинок
        y = [{k: v.to(device) for k, v in t.items()} for t in y] # таргет в список из словарей
        
        optimizer.zero_grad()
        
        output = model(x, y) # output с ошибками
        
        loss_sum = sum(loss for loss in output.values())
        
        loss_sum.backward()
        
        optimizer.step()

In [23]:
#!pip install torchmetrics


@torch.inference_mode()
def evaluate(model, loader):
    model.eval()
    
    metric = MeanAveragePrecision() # наша метрика
    
    for x, y in tqdm(loader, desc='Evaluation'):
        x = list(_.to(device).float() for _ in x)
        
        output = model(x)
        
        output = [{k: v.cpu() for k, v in t.items()} for t in output]
        
        metric.update(output, y) 
        
    return metric.compute()['map']

In [24]:
def whole_train_valid_cycle(model, loader, optimizer, title, valid_loader, num_epoch=100):
    batch = next(iter(valid_loader))
    
    m = 0
    epoch = 0
    
    while m < 0.6 or epoch != num_epoch:
        train(model, loader, optimizer)
        
        clear_output()
        
        m = evaluate(model, loader)
        v = evaluate(model, valid_loader)
        print(f'map score: {m} for train and map score: {v} for valid')
        epoch += 1
        
        visualize(model, batch)

In [25]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [26]:
from torch.optim import Adam


model = get_detection_model().to(device)

optimizer = Adam([p for p in model.parameters() if p.requires_grad], lr=1e-4)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_MobileNet_V3_Large_320_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_mobilenet_v3_large_320_fpn-907ea3f9.pth
100%|██████████| 74.2M/74.2M [00:01<00:00, 49.0MB/s]


In [27]:
modelv = get_detection_model().to(device)

optimizerv = Adam([p for p in model.parameters() if p.requires_grad], lr=1e-4)

In [28]:
from IPython.display import clear_output


#whole_train_valid_cycle(model, train_loader, optimizer,'first faster-rcnn', valid_loader, 100)

In [29]:
#whole_train_valid_cycle(modelv, train_loaderv, optimizerv,'first faster-rcnn', valid_loaderv, 100)

In [30]:
# !pip uninstall torchmetrics
# !pip uninstall pycocotools
# !pip uninstall faster-coco-eval
# !pip uninstall torchmetrics[detection]==0.8.0

# # from torchmetrics.detection.mean_ap import MeanAveragePrecision

# # m = MeanAveragePrecision()

### YOLO

In [6]:
!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/ad/1e/62ed77dbb887a963c788c61dc6e740c1d16462fcdd37acf247099f1a911b/ultralytics-8.0.229-py3-none-any.whl.metadata
  Using cached ultralytics-8.0.229-py3-none-any.whl.metadata (32 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.2/663.2 kB 10.7 MB/s eta 0:00:00a 0:00:01


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.229 🚀 Python-3.10.12 torch-2.0.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5310.5/8062.4 GB disk)


In [8]:
from ultralytics import YOLO

In [33]:
from ultralytics import YOLO


modelYM = YOLO('yolov8m.pt')


results = modelYM.train(
    data='/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/data.yaml',
    imgsz=640,
    epochs=50,
    batch=8,
    name='yolo8m_custom'
)

100%|██████████| 49.7M/49.7M [00:00<00:00, 225MB/s]


Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/input/forall/detectInsulators.v2i.yolov7pytorch/data.yaml, epochs=50, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo8m_custom, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 755k/755k [00:00<00:00, 14.9MB/s]
2023-12-20 20:06:57,432	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-12-20 20:06:57,966	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.9MB/s]


...

      50/50      4.55G     0.9052      0.569       1.21          6        640: 100%|██████████| 53/53 [00:17<00:00,  3.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.90it/s]

                   all         40        103      0.822      0.694      0.702      0.441



50 epochs completed in 0.281 hours.
Optimizer stripped from runs/detect/yolo8m_custom/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolo8m_custom/weights/best.pt, 52.0MB

Validating runs/detect/yolo8m_custom/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]


                   all         40        103      0.799      0.734      0.723      0.444
                   BAD         40         11      0.694       0.62      0.556      0.286
                  GOOD         40         92      0.904      0.848      0.889      0.602
Speed: 2.1ms preprocess, 11.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/yolo8m_custom


In [36]:
torch.save(modelYM.state_dict(), 'yolov8m1.pt')

In [13]:
!ls '/kaggle/working/'

In [37]:
torch.save(modelYM2.state_dict(), 'yolov8m2.pt')

In [35]:
modelYM2 = YOLO('yolov8m.pt')


results2 = modelYM2.train(
    data='/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/data.yaml',
    imgsz=640,
    epochs=50,
    batch=8,
    name='yolo8m_custom2'
)

Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/data.yaml, epochs=50, time=None, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo8m_custom22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True,

train: Scanning /kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train/labels... 513 images, 0 backgrounds, 0 corrupt: 100%|██████████| 513/513 [00:00<00:00, 753.96it/s]

train: WARNING ⚠️ Cache directory /kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/train is not writeable, cache not saved.
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/valid/labels... 41 images, 0 backgrounds, 0 corrupt: 100%|██████████| 41/41 [00:00<00:00, 830.41it/s]

val: WARNING ⚠️ Cache directory /kaggle/input/forall/brokeInsulators.v4i.yolov7pytorch/valid is not writeable, cache not saved.


...

      50/50      6.13G     0.9486     0.4382      1.316          1        640: 100%|██████████| 65/65 [00:21<00:00,  3.06it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  4.73it/s]

                   all         41         63      0.939      0.984      0.974      0.632



50 epochs completed in 0.334 hours.
Optimizer stripped from runs/detect/yolo8m_custom22/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolo8m_custom22/weights/best.pt, 52.0MB

Validating runs/detect/yolo8m_custom22/weights/best.pt...
Ultralytics YOLOv8.0.228 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.56it/s]


                   all         41         63      0.925      0.975      0.974      0.645
Speed: 3.0ms preprocess, 13.7ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/yolo8m_custom22
